<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from random import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

RND_SEED = 12345

In [2]:
data = pd. read_csv('/datasets/insurance.csv')

In [3]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Пропусков в данных нет, серьезных аномалий, исходя из таблицы, не видно.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** При умножении на обратимую матрицу качество модели не меняется

**Обоснование:** умножаем матрицу признаков на обратимую матрицу P:
$$
wp = ((XP)^T XP)^{-1} (XP)^T y
$$
раскрываем скобки
$$
wp = (P^T X^T X P)^{-1} P^T X^T y
$$
представим $X^T X$ как множитель, дораскроем скобки
$$
wp = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$(P^T)^{-1} P^T$ является единичной матрицей. Подставим $w$:
$$
wp = P^{-1} w
$$
следственно:
$$
ap = XPwp = XPP^{-1} w = XEw = Xw = a
$$

## Алгоритм преобразования

**Алгоритм**

1. создать случайную матрицу
2. проверить ее на обратимость
3. умножить признаки на полученную матрицу
4. сравнить метрики R2 исходных и преобразованных данных.

In [5]:
class MatrixCoder:
    _key = []
    def __create_key__(self, data):
        det = 0
        while det == 0:
            self._key = np.random.randint(1000, size=(data.shape[1], data.shape[1]))
            det = np.linalg.det(self._key)
            
    def transform(self, data):
        if len(self._key) == 0:
            self.__create_key__(data)
        self.data = data.dot(self._key)
        self.data.columns = data.columns
        return self.data
    
    def decode(self, data):
        self.data = np.round(data.dot(np.linalg.inv(self._key)))
        self.data.columns = data.columns
        return self.data
    
    def show_key(self):
        return self._key
    

**Обоснование**

Шифрующая матрица должна иметь размерность равную количеству признаков шифруемых данных, являться квадратной и обратимой (детерминант != 0). Таким образом сгенерированная матрица будет перемножать данные сохраняя размерность, а умножение на обратную матрицу приведет данные в исходный вид. 

## Проверка алгоритма

Делим на таргет и фичи, делаем выборки данных

In [6]:
data_X, data_y = data.drop(['Страховые выплаты'], axis=1), data['Страховые выплаты']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Страховые выплаты'], axis=1), 
                                                    data['Страховые выплаты'], 
                                                    test_size=0.35, 
                                                    random_state=RND_SEED)

Функция обучает линейную регрессию и возвращает R2

In [8]:

def check_model(X_train=X_train, X_test=X_test):
    model = LinearRegression() 
    model.fit(X_train, y_train)
    return r2_score(y_test, model.predict(X_test))

In [9]:
# получаем R2 для чистых данных
r2_clean_data = check_model()

In [10]:
# кодируем признаки и созраняем для них R2 
mc = MatrixCoder()
X_train_coded, X_test_coded = mc.transform(X_train), mc.transform(X_test)
r2_coded_data = check_model(X_train_coded, X_test_coded)

In [11]:
print("R2 чистых данных = {}, \nR2 кодированных данных = {}, \nразница = {}"
      .format(r2_clean_data, r2_coded_data, r2_clean_data - r2_coded_data))

R2 чистых данных = 0.4270038468669354, 
R2 кодированных данных = 0.4270038468669235, 
разница = 1.1879386363489175e-14


Погрешность появляется после 14 знака, разница незначительна.

In [12]:
X_train_coded.head(5)

,Пол,Возраст,Зарплата,Члены семьи
160,26546446.0,4936351.0,9786697.0,32365152.0
1765,36524622.0,6786855.0,13462467.0,44532190.0
196,29422648.0,5464902.0,10838220.0,35870694.0
2023,37557078.0,6975852.0,13840533.0,45792042.0
807,39033920.0,7248864.0,14383104.0,47591968.0


Данные действительно искажаются и понять их сложно.

In [13]:
X_train.head(5)

,Пол,Возраст,Зарплата,Члены семьи
160,0,32.0,35900.0,3
1765,0,38.0,49400.0,1
196,0,18.0,39800.0,4
2023,1,34.0,50800.0,0
807,0,32.0,52800.0,0


In [14]:
(mc.decode(X_train_coded)).head(5)

,Пол,Возраст,Зарплата,Члены семьи
160,-0.0,32.0,35900.0,3.0
1765,-0.0,38.0,49400.0,1.0
196,-0.0,18.0,39800.0,4.0
2023,1.0,34.0,50800.0,0.0
807,-0.0,32.0,52800.0,0.0


Декодирование проходит успешно

In [15]:
mc.show_key()

array([[238,  34,  85, 876],
       [460, 477, 672, 599],
       [739, 137, 272, 901],
       [542, 929, 131,  28]])

Ключ, которым выполняются преобразования.

## Вывод ##

По результатам проведенного исследования доказано, что при умножении матрицы признаков на случайную обратимую матрицу можно визуально исказить данные, не потеряв качества модели обучения. Для решения задачи написан алгоритм и составлен класс, который выполняет поставленную задачу. Его можно применять в дальнейших проектах.